Import packages

In [57]:
import re
from glob import glob
import numpy as np
import os

Import the log files to parse for and name them for the plot

In [58]:
# List of filenames to process
log_files = glob("neutral/PTH_polythiophene/01n_RuCap_PTH_NICS_*.log")

['neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_0.log', 'neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_1.log', 'neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_2.log']


Sort by distance from Ru atom

In [59]:
def find_ru_coordinates(logfile):
    with open(logfile, 'r') as file:
        log_text = file.read()

    # Define regex pattern to extract Ru coordinates
    pattern = r'Ru\s+([-+]?\d*\.\d+\s+[-+]?\d*\.\d+\s+[-+]?\d*\.\d+)'

    # Search for Ru coordinates
    match = re.search(pattern, log_text)
    if match:
        coordinates = [float(coord) for coord in match.group(1).split()]
        return np.array(coordinates)
    else:
        return None

def distance_from_origin(coordinates):
    return np.linalg.norm(coordinates)

# Dictionary to store distances for each file
distances = {}

# Iterate through log files, extract Ru coordinates, calculate distances, and store in dictionary
for logfile in log_files:
    ru_coordinates = find_ru_coordinates(logfile)
    if ru_coordinates is not None:
        dist = distance_from_origin(ru_coordinates)
        distances[logfile] = dist

# Sort log files based on numerical prefix and distances
sorted_files = [file[0] for file in sorted(distances.items(), key=lambda x: (int(re.findall(r'\d+', os.path.basename(x[0]))[0]), x[1]))]

# Print sorted log files
for file in sorted_files:
    print(os.path.basename(file))

01n_RuCap_PTH_NICS_2.log
01n_RuCap_PTH_NICS_0.log
01n_RuCap_PTH_NICS_1.log
['neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_2.log', 'neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_0.log', 'neutral/PTH_polythiophene\\01n_RuCap_PTH_NICS_1.log']


Regex to find the Banquo atom NICS readout and print zz_values

In [60]:
# Define the expression to match the log line

# regex_pattern = r"\s*\d+\s+Bq\s+Isotropic\s+=\s+-?\d+\.\d+\s+Anisotropy\s+=\s+\d+\.\d+\n\s+XX=\s+-?\d+\.\d+\s+YX=\s+-?\d+\.\d+\s+ZX=\s+-?\d+\.\d+\n\s+XY=\s+-?\d+\.\d+\s+YY=\s+-?\d+\.\d+\s+ZY=\s+-?\d+\.\d+\n\s+XZ=\s+-?\d+\.\d+\s+YZ=\s+-?\d+\.\d+\s+ZZ=\s+(-?\d+\.\d+)"
regex_pattern = r"\s*\d+\s+Bq\s+Isotropic\s+=\s+-?\d+\.\d+\s+.*\n\s+.*\n\s+.*\n\s+.*\n\s+.*\n\s+.*\n\s+.*\n\s+ZZ=\s+(-?\d+\.\d+)"

zz_list = []

# Loop over each file name, extract the data and plot it
for i, file in enumerate(sorted_files):

    # Open the log file and read its contents
    with open(file, "r") as f:
        log_contents = f.read()

    # Find all instances of the log line using the expression
    matches = re.findall(regex_pattern, log_contents)

    # Extract the ZZ values from the matches and store them in a list
    zz_values = [-float(match) for match in matches]

    zz_list.extend(zz_values)

print(zz_list)
print([zz_list[0], zz_list[-1]])
del zz_list[0]
del zz_list[-1]
print(zz_list)
print(round(sum(zz_list)/len(zz_list), 4))

[-21.3152, -17.396, -21.1979]
[-21.3152, -21.1979]
[-17.396]
-17.396
